**EXTRAIR OS DADOS**

Importar bibliotecas necessárias

In [ ]:
import pandas as pd # Para manipulação de databeses e dataframes

Obter os caminhos dos arquivos csv

In [ ]:
path_cargos = '../data_raw/cargos.csv'
path_estruturas = '../data_raw/estruturas.csv'
path_funcionarios = '../data_raw/funcionarios.csv'

Ler arquivos CSV como DataFrames

In [ ]:
# Usar encoding:'utf-8' para garantir que os dados sigam os padrões de acentuação brasileiro.

df_cargos = pd.read_csv(path_cargos, encoding='utf-8')
df_estruturas = pd.read_csv(path_estruturas, encoding='utf-8')
df_funcionarios = pd.read_csv(path_funcionarios, encoding='utf-8')

In [ ]:
# Validar os dados extraídos
df_funcionarios

**TRANSFORMAR OS DADOS**

Transformar as colunas de data para o tipo 'datetime'

In [ ]:
# Cria uma lista com todos os dataframes
df_list = [df_cargos, df_estruturas, df_funcionarios]

In [ ]:
# Verifica o tipo atual das colunas de data
print(df_cargos['start_date'][0])
type(df_cargos['start_date'][0])

In [ ]:
# Cria uma lista com todos os nomes das colunas de data
date_columns = ['start_date', 'end_date', 'data_admissao', 'data_desligamento']

# Transforma o tipo de todas as colunas de data em todos os dataframes para o formato 'datetime'
for df in df_list:

    # Obter todas os nomes das colunas do dataframe atual
    df_columns = df.columns.tolist()

    for column in df_columns:
        # Verifica se o nome da atual coluna possui na lista de 'date_columns'
        if column in date_columns:
            # Transforma a coluna atual para o tipo 'datetime'
            df[column] = pd.to_datetime(df[column], errors='coerce')


In [ ]:
# Confirma o novo tipo das colunas de data
df_cargos['start_date'][0]

Unir as tabelas "funcionarios" e "cargos"

In [ ]:
df_merged = pd.merge(df_funcionarios, df_cargos, on='id_cargo', how='left') # LEFT JOIN para manter "funcionarios", mesmo que não exista um "cargo" correspondente

Filtrar "data_admissao" e "data_desligamento" para estar entre "start_date" e "end_date"

In [ ]:
# Obtém todos NaT de "data_desligamento" e substitui por "end_date" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_merged.loc[df_merged['data_desligamento'].isna(), 'data_desligamento'] = df_merged['end_date']


In [ ]:
# Filtra o novo dataframe unido
df_filtered = df_merged[
    (df_merged['data_admissao'] >= df_merged['start_date']) &
    (df_merged['data_desligamento'] <= df_merged['end_date'])
]

In [ ]:
# Valida o número de linhas do antigo e o novo dataframe filtrado
print("Total de linhas de df_merged: " + str(df_merged.__len__()))
print("Total de linhas de df_filtered: " + str(df_filtered.__len__()))

In [ ]:
# Confirma a nova tabela filtrada (funcionarios e cargos)
df_filtered

Unir as tabelas anteriores e a tabela "estruturas"

In [ ]:
df_merged_all = pd.merge(df_filtered, df_estruturas, on='id_estrutura', how='left') # LEFT JOIN para manter os dados anteriores

In [ ]:
# Renomeia a colunas start e end para uma melhor manipulação posterior
df_merged_all = df_merged_all.rename(columns={
    'start_date_x': 'start_date_cargo',
    'end_date_x': 'end_date_cargo',
    'start_date_y': 'start_date_estrutura',
    'end_date_y': 'end_date_estrutura'
})

Filtra "data_admissao" e "data_desligamento" para estar entre "start_date_estrutura" e "end_date_estrutura"

In [ ]:
# Filtrar o dataframe com todas as tabelas unidas
df_filtered_all = df_merged_all[
    (df_merged_all['data_admissao'] >= df_merged_all['start_date_estrutura']) &
    ((df_merged_all['data_desligamento'] <= df_merged_all['end_date_estrutura']))
]

In [ ]:
print("Tamanho da nova tabela filtrada: " + str(df_filtered_all.__len__()))

Remove as colunas "start_date" e "end_date"

In [ ]:
# Remove colunas
df_formated = df_filtered_all.drop(axis=1, columns=[
    'start_date_cargo', 'end_date_cargo', 'start_date_estrutura', 'end_date_estrutura'
])

Reorganizar as colunas

In [ ]:
# Reoganiza as colunas na ordem desejada (Para melhor visualização do usuário final)
df_formated = df_formated[['id_funcionario', 'data_admissao', 'data_desligamento', 'estado', 'id_cargo', 'nome_cargo', 'id_estrutura', 'nome_estrutura']]

In [ ]:
# Obter a tabela final com todas as tabelas unidas e filtradas
df_formated

**CARREGAR OS DADOS**

Carrega a tabela formatada para o formato csv no caminho de destino

In [ ]:
end_path = '../data_processed/'
file_name = 'tabela_formatada.csv'

df_formated.to_csv(end_path + file_name, index=False, encoding='utf-8')

**OBTER INDICADORES**

Preparação dos dados dos indicadores

In [ ]:
# Cria duas novas colunas com o total de "data_admissao" and "data_desligamento" por mês
df_formated['mes_admissao'] = df_formated['data_admissao'].dt.to_period('M')
df_formated['mes_desligamento'] = df_formated['data_desligamento'].dt.to_period('M')

Obter o total de funcionários contratados e desligados por mês

In [ ]:
# Obter a tabela (serie) com o total de admissão por mês
s_admitidos = df_formated['mes_admissao'].value_counts().sort_index()
s_admitidos = s_admitidos.rename('admitidos') # Da um nome a tabela (serie)
s_admitidos

In [ ]:
# Obter a tabela (serie) com o total de desligamento por mês
s_desligados = df_formated['mes_desligamento'].value_counts().sort_index()
s_desligados = s_desligados.rename('desligados') # Da um nome a tabela (serie)
s_desligados

Obter o total de funcionários ativos por mês

In [ ]:
# Obter o primeiro e o último mês registrado na tabela
first_date = df_formated['mes_admissao'].min()
last_date = df_formated['mes_desligamento'].max()

print('Data inicial: ' + str(first_date))
print('Data final: ' + str(last_date))

# Obter todos os meses entre "first_date" e "last_date"
all_months = pd.period_range(first_date, last_date, freq='M')

print("Total de meses entre as duas: " + str(all_months.__len__()))

In [ ]:
# Obter a tabela (serie) com o total de funcionários ativos por mês
s_ativos = pd.Series(index=all_months, dtype='Int64', name='ativos')

for mes in all_months:
    s_ativos[mes] = df_formated[
        (df_formated['mes_admissao'] <= mes) &
        (df_formated['mes_desligamento'] >= mes)
    ].shape[0]

s_ativos = s_ativos.rename_axis('mes')
s_ativos

Criar um único dataframe com todos os indicadores

In [ ]:
# Cria uma lista com todas as tabelas (series) de indicadores
series_list = [s_ativos, s_admitidos, s_desligados]

# Transforma as series em um dataframe (Unidos pelos meses)
df_indicadores = pd.concat(series_list, axis=1)

# Transforma todos os valores 'NaN' em 0
df_indicadores = df_indicadores.fillna(0).astype({
    col: 'Int64' for col in df_indicadores.select_dtypes(include='float').columns # Transforma todas as colunas do tipo 'float' no tipo 'int'
    })

# Transforma o index em uma coluna
i_column_name = 'mes'
df_indicadores = df_indicadores.reset_index(names=i_column_name)

In [ ]:
# Obter a tabela final de indicadores
df_indicadores

**CARREGAR TABELA INDICADORES**

Carrega a tabela de indicadores formatada para o formato csv no caminho de destino

In [ ]:
end_path = '../data_processed/'
file_name = 'indicadores.csv'

df_indicadores.to_csv(end_path + file_name, index=False, encoding='utf-8')